In [147]:
import graphlab as gl
import pickle
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
data_items = pickle.load(open("/Users/marvinbertin/Github/family_style_chat_bot/data/user_by_cuisine_by_dish_ratings.pkl", 'rb'))
data_cuisine = pickle.load(open("/Users/marvinbertin/Github/family_style_chat_bot/data/user_by_cuisine_ratings.pkl", 'rb'))

In [3]:
df_cuisine = pd.DataFrame(data_cuisine)
df_cuisine.head()

,user_id,item_id,rating
0,Brian,China,2
1,David,China,0.5
3,Jon,China,4
4,Marvin,China,0.5
6,Anne,China,2.5


In [194]:
df_cuisine.pivot(index='user_id', columns="item_id")

rating                                                            \
item_id   China France India Indonesia Italy Japan Mexico Pizza Singapore   
user_id                                                                     
Anne        2.5    1.5     4      None     4     4    1.5     5         4   
Bob        None   None  None      None   2.5     2      3   3.5       2.5   
Brian         2    3.5  None      None   4.5     5   None  None         5   
David       0.5   None  None         5   3.5   1.5   None  None       1.5   
Eugene     None   None  None       0.5     2  None   None     1         2   
Jon           4      1     4       2.5   0.5     3   None  None         5   
Leah          4   None     1      None  None  None    2.5     5       1.5   
Marvin      0.5   None   4.5       4.5  None  None   None     2      None   
Michelle    4.5      1  None         5   4.5  None   None   1.5      None   
Samatha       2      3     5      None  None   2.5   None  None       3.5   
Sarah       0.5   None   1.5      None  None  None   None     1         3   

                           
item_id  Thailand Vietnam  
user_id                    
Anne         None       5  
Bob          None     1.5  
Brian         3.5    None  
David         3.5       2  
Eugene          1    None  
Jon          None     2.5  
Leah         None    None  
Marvin        1.5    None  
Michelle     None     1.5  
Samatha       4.5     2.5  
Sarah           4     2.5

In [4]:
data_items.keys()

['Italy',
 'Mexico',
 'Indonesia',
 'India',
 'Singapore',
 'France',
 'Vietnam',
 'China',
 'Thailand',
 'Japan',
 'Pizza']

In [195]:
class group_recommender(object):
    def __init__(self, cuisine_sf, dict_cuisine_items):
        self.cuisine = gl.SFrame(cuisine_sf)
        self.cuisine_items = dict_cuisine_items
    
    def recommend(self, group_list):
        group_name = "_".join(group_list)
        
        sf_avg_user = self.cuisine.filter_by(group_list, "user_id") \
                                  .groupby(key_columns='item_id',
                                           operations={'rating': gl.aggregate.MEAN('rating')})  
        
        sf_avg_user.add_column(gl.SArray([group_name] * len(sf_avg_user)), "user_id")
        print sf_avg_user
        
        sf_new = self.cuisine.append(sf_avg_user)
        
        model = gl.recommender.create(sf_new, target='rating')
    
        results = model.recommend([group_name], exclude_known=False)
        print results
        result_cuisine = results["item_id"][0]
            
        sf_items = gl.SFrame(self.cuisine_items[result_cuisine])
        model_items = gl.recommender.create(sf_items, target='rating')
        
        results_items = model_items.recommend(group_list, exclude_known=False, k = 2)
        print results_items
        
        
        if result_cuisine == "Pizza":
            group_size = len(group_list)
            num_pizza = int(group_size / 1.5)
            item_results = [item for item, count in Counter(results_items["item_id"]).most_common()][:num_pizza]
            return ("Everyone", item_results)
        
        else:
            name_items = results_items[["user_id", "item_id"]].to_numpy().tolist()

            recommendations = []
            for name in group_list:
                recommendations.append((name,
                                        map(lambda (n, meal): meal,
                                            filter(lambda (n, meal): n == name , name_items))))
            return recommendations

In [206]:
group_list = np.random.choice(df_cuisine["user_id"].unique(), size = 2, replace=False)

print group_list
model = group_recommender(df_cuisine, data_items)
result = model.recommend(group_list)

['Brian' 'Marvin']
+-----------+--------+--------------+
|  item_id  | rating |   user_id    |
+-----------+--------+--------------+
|   China   |  1.25  | Brian_Marvin |
|   Japan   |  5.0   | Brian_Marvin |
|   France  |  3.5   | Brian_Marvin |
|   Pizza   |  2.0   | Brian_Marvin |
| Indonesia |  4.5   | Brian_Marvin |
|   Italy   |  4.5   | Brian_Marvin |
| Singapore |  5.0   | Brian_Marvin |
|   India   |  4.5   | Brian_Marvin |
|  Thailand |  2.5   | Brian_Marvin |
+-----------+--------+--------------+
[9 rows x 3 columns]



Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 79 observations with 12 users and 11 items.

Data prepared in: 0.005661s

...

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.458871

Final training RMSE: 0.2922

+---------+-------------------------------+---------------+------+
| user_id |            item_id            |     score     | rank |
+---------+-------------------------------+---------------+------+
|  Brian  |        Singapore Satay        | 4.81741923142 |  1   |
|  Brian  |        Singapore Laksa        | 2.48972246099 |  2   |
|  Marvin |    Sesame Oil Chicken Wings   | 4.91785606045 |  1   |
|  Marvin | Singapore Noodle Curry Shrimp | 3.70050325293 |  2   |
+---------+-------------------------------+---------------+------+
[4 rows x 4 columns]



In [207]:
result

[('Brian', ['Singapore Satay', 'Singapore Laksa']),
 ('Marvin', ['Sesame Oil Chicken Wings', 'Singapore Noodle Curry Shrimp'])]